# Chapter 3.4
## Sire Model
Abhilash Dhal

Consider the following data set for the pre-weaning gain (WWG) of beef calves (calves assumed to be reared under the same management conditions). The objective is to estimate sex effects and predict breeding values for sires only (1, 3 and 4). The data set is obtained from the example 3.3

Assume $\sigma^2_a = 20$ and $\sigma^2_e = 40$, $\sigma^2_s = 0.25(\sigma^2_a)$, $\alpha = \frac{\sigma^2_e}{\sigma^2_s}$


In [1]:
using JWAS, DataFrames

In [13]:
data = DataFrame(sexofprogeny=["male","female","female","male","male"], sire=[1,3,1,4,3], sireofsire=[missing,missing,missing,1,missing], damofsire=[missing,missing,missing,missing,missing], WWG=[4.5,2.9,3.9,3.5,5.0])

,sexofprogeny,sire,sireofsire,damofsire,WWG
,String,Int64,Int64⍰,Missing,Float64
1,male,1,missing,missing,4.5
2,female,3,missing,missing,2.9
3,female,1,missing,missing,3.9
4,male,4,1,missing,3.5
5,male,3,missing,missing,5.0


The model to describe the observations is:

<center>$y_{ij} = p_i + a_j +e_{ij}$</center>

or

<center>$Y = Xb + Zs + e$</center>

where: 

$y_{ij} = $ the WWG of the $j$th sire's progeny for the $i$th sex

$p_i = $ the fixed effect of the $i$th sex of the sire's progeny

$a_j = $ the random effect of the $j$th sire

$e_{ij} = $ the random error effect

In [14]:
y = [4.5, 2.9, 3.9, 3.5, 5.0]

X = [1 0; 0 1; 0 1; 1 0; 1 0]
b = ["male", "female"];


The design matrix for fixed effects(sex for sire's progeny) is the same as the random animal effects model.

However, in the sire model since we consider the random effects of the sire, the Z matrix will be of dimensions(5,3)

In [15]:
Z = [1 0 0 
     0 1 0 
     1 0 0 
     0 0 1 
     0 1 0]
s = ["sBV1", "sBV3", "sBV4"];

In [16]:
xpx =  X'X;
xpz = X'Z;
zpx = Z'X;
zpz = Z'Z;
xpy = X'y;
zpy = Z'y

3-element Array{Float64,1}:
 8.4
 7.9
 3.5

In [60]:
model_equation = "WWG = sexofprogeny + sire"

"WWG = sexofprogeny + sire"

In [61]:
R = 55.0 ;
G = 0.25(20.0);

In [62]:
model1 = build_model(model_equation,R);
set_random(model1,"sire",G);

In [63]:
outputMCMCsamples(model1,"sire");

In [64]:
out2=runMCMC(model1, data, chain_length=50000,output_samples_frequency=100,outputEBV=true);

A Linear Mixed Model was build using model equations:

WWG = sexofprogeny + sire

Model Information:

Term            C/F          F/R            nLevels
sexofprogeny    factor       fixed                2
sire            factor       random               3

MCMC Information:

methods                        conventional (no markers)
chain_length                                  50000
burnin                                            0
estimateScale                                 false
starting_value                                false
printout_frequency                            50001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                             true
update_priors_frequency                           0

Hyper-parameters Information:

random effect variances (sire):                [5.0]
residual variances:                          55.000

Degree of freedom for hyper-parameters:

residual variances

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [65]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 2 entries. Keys:
  "Posterior mean of residual variance"
  "Posterior mean of location parameters"

In [66]:
JWAS_sol = out2["Posterior mean of location parameters"][:, 4]

5-element Array{Any,1}:
  4.315090753598826  
  3.335851144862729  
  0.09417901945239061
  0.0367033041856141 
 -0.145897248320225  

In [70]:
JWAS_sol[3:5]

3-element Array{Any,1}:
  0.09417901945239061
  0.0367033041856141 
 -0.145897248320225  

In [71]:
MME_Sol = [0.022 0.014 -0.043]'

3×1 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
  0.022
  0.014
 -0.043

In [73]:
cor(JWAS_sol[3:5], MME_Sol)

1×1 LinearAlgebra.Transpose{Float64,Array{Float64,1}}:
 0.9930205364328477